## Create IAM role for Lambda function and attach Sagemaker priviliges to it


In [8]:
import boto3
import json

iam_client = boto3.client('iam')

role_policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "",
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

In [9]:
iam_rolename = "LambdaHFServerlessRole"

response = iam_client.create_role(
  RoleName=iam_rolename,
  AssumeRolePolicyDocument=json.dumps(role_policy),
)

In [10]:
response = iam_client.attach_role_policy(RoleName=iam_rolename, PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess')

## Create Lambda Function

In [5]:
endpoint_name = 'patient-doctor-text-classifier-endpoint'

lambda_code = f"""
import boto3
import json

ENDPOINT_NAME = "{endpoint_name}"
runtime= boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
  print("Received event: " + json.dumps(event, indent=4))
  
  payload = {"inputs": event}
  response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                    ContentType='application/json',
                                    Body=json.dumps(payload))
  
  result = json.loads(response['Body'].read().decode())
  print(f"Predicted result: {result}")

  return result
"""

In [7]:
from zipfile import ZipFile
filename = "lambda_function"

with open(f"{filename}.py", "w") as f:
  f.write(lambda_code)

with ZipFile(f'{filename}.zip', 'w') as myzip:
  myzip.write(f'{filename}.py')

In [ ]:
import boto3
lambda_function_name = "SagemakerInvokeEndpointFunction"

lambda_client = boto3.client('lambda')

with open('lambda_function.zip', 'rb') as f:
    zipped_code = f.read()
  
iam_client = boto3.client('iam')
iam_rolename = "SagemakerInvokeEndpointRole"
role = iam_client.get_role(RoleName=iam_rolename)

response = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.9',
    Role=role['Role']['Arn'],
    Handler=f'lambda_function.lambda_handler',
    Code=dict(ZipFile=zipped_code),
    Timeout=300,
)